In [1]:
import numpy as np
import cv2
from PIL import ImageGrab

### Mèthode de la moyenne récursive

In [27]:
def motion_detector_moyen(alpha):
    frames = 0
    prev_frame = None
    prepared_frame = None
    video_capture = cv2.VideoCapture('./MV_TRIMMED.mp4')
    frame_count = video_capture.get(7)
    #print(frame_count)
    #print(video_capture.isOpened())

    while (video_capture.isOpened() and frames < frame_count):
        frames += 1
        # img_brg = np.array(ImageGrab.grab())
        # img_rgb = cv2.cvtColor(img_brg, cv2.COLOR_BGR2RGB)
        ret, img_brg = video_capture.read()
        img_rgb = cv2.cvtColor(img_brg, cv2.COLOR_BGR2RGB)

        if((frames % 2) == 0):
            # convert to grayscale
            prepared_frame = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
            # smooth the image with gaussian blur
            prepared_frame = cv2.GaussianBlur(prepared_frame, (5, 5), 0)
        
        if(prev_frame is None):
            prev_frame = prepared_frame
            continue
        
        # calculate the difference
        m = alpha * prepared_frame + (1 - alpha) * prev_frame
        m = np.uint8(m)
        diff_frame = cv2.absdiff(m, prepared_frame)
        prev_frame = prepared_frame

        # add dilation
        diff_frame = cv2.dilate(diff_frame, np.ones((5,5)), iterations=2)
        # add threshold
        _, diff_frame = cv2.threshold(diff_frame, 20, 255, cv2.THRESH_BINARY)

        # find and draw contours
        contours, _ = cv2.findContours(diff_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(img_rgb, contours, -1, (0, 255, 0), 2)

        cv2.imshow('Detection de mouvement', img_rgb)

        if(cv2.waitKey(1) & 0xFF == ord('q')):
            video_capture.release()
            break

### Méthode de Richefeu 

In [35]:
def motion_detector_richefeu(alpha):
    frames = 0
    #prev_frame = None
    prepared_frame = None
    M = None
    m = None
    video_capture = cv2.VideoCapture('./MV_TRIMMED.mp4')
    frame_count = video_capture.get(7)
    #print(frame_count)
    #print(video_capture.isOpened())

    while (video_capture.isOpened() and frames < frame_count):
        frames += 1
        # img_brg = np.array(ImageGrab.grab())
        # img_rgb = cv2.cvtColor(img_brg, cv2.COLOR_BGR2RGB)
        ret, img_brg = video_capture.read()
        img_rgb = cv2.cvtColor(img_brg, cv2.COLOR_BGR2RGB)

        if((frames % 2) == 0):
            # convert to grayscale
            prepared_frame = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
            # smooth the image with gaussian blur
            prepared_frame = cv2.GaussianBlur(prepared_frame, (5, 5), 0)
        
        if(M is None and m is None):
            #prev_frame = prepared_frame
            M = prepared_frame
            m = prepared_frame
            continue
        
        # calculate the difference
        M = alpha * prepared_frame + (1 - alpha) * np.maximum(prepared_frame, M)
        M = np.uint8(M)
        m = alpha * prepared_frame + (1 - alpha) * np.minimum(prepared_frame, m)
        m = np.uint8(m)
        diff_frame = cv2.subtract(M, m)
        #prev_frame = prepared_frame

        # add dilation
        diff_frame = cv2.dilate(diff_frame, np.ones((5,5)), iterations=2)
        # add threshold
        _, diff_frame = cv2.threshold(diff_frame, 20, 255, cv2.THRESH_BINARY)

        # find and draw contours
        contours, _ = cv2.findContours(diff_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(img_rgb, contours, -1, (0, 255, 0), 2)

        cv2.imshow('Detection de mouvement', img_rgb)

        if(cv2.waitKey(1) & 0xFF == ord('q')):
            video_capture.release()
            break

### Tune the parameter $\alpha$ to get variant results

In [34]:
motion_detector_moyen(0.5)
cv2.destroyAllWindows()

In [38]:
motion_detector_richefeu(0.5)
cv2.destroyAllWindows()